In [2]:
import yaml
import sys, os
sys.path.append(os.path.abspath("..")) 
from Model.BusinessGlossaryModel import BusinessGlossaryModel 
from Service.DataHubGlossaryImporter import DataHubGlossaryImporter
import Service.Utils as utils



full_path = "../Template/template_business_glossary.yaml"

if full_path.endswith(".yaml") : 
    
    with open(full_path) as stream:
        data = yaml.safe_load(stream)
        dataset = BusinessGlossaryModel(**data)
        importer  = DataHubGlossaryImporter(dataset)

INFO:Service.Utils:Created node: urn:li:glossaryNode:operation
INFO:Service.Utils:Created node: urn:li:glossaryNode:operation.general
INFO:Service.Utils:Created term: urn:li:glossaryTerm:Driver8
INFO:Service.Utils:Upserted group urn:li:corpGroup:Ops_team
INFO:Service.Utils:Upserted group urn:li:corpGroup:BITEAM
INFO:Service.Utils:Owner urn:li:corpGroup:BITEAM, type TECHNICAL_OWNER added to dataset urn:li:glossaryTerm:Driver8
INFO:Service.Utils:Owner urn:li:corpGroup:Ops_team, type BUSINESS_OWNER added to dataset urn:li:glossaryTerm:Driver8


 Linked column '' in dataset 'base_api.test15' to glossary term 'Driver8'


In [4]:
test =  "[mysql].[127.0.0.1,3306].[base_api.test15].[columnName2]" 

technical_dataset= test.split('].[')

technical_dataset = [item.replace(']', '').replace('[', '')  for item in technical_dataset]

print(technical_dataset)

['mysql', '127.0.0.1,3306', 'base_api.test15', 'columnName2']


In [ ]:
import requests
import yaml
import sys, os
sys.path.append(os.path.abspath("..")) 
from Model.BusinessGlossaryModel import BusinessGlossaryModel 
from Service.DataHubGlossaryImporter import DataHubGlossaryImporter


full_path = "../Template/template_business_glossary.yaml"

if full_path.endswith(".yaml") : 
    
    with open(full_path) as stream:
        data = yaml.safe_load(stream)
        test = requests.post ("http://127.0.0.1:8000/", data=data)